In [ ]:
!pip install transformers peft accelerate datasets trl bitsandbytes wandb

In [ ]:
!pip install transformers --upgrade

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset, Dataset

rawDataset = load_dataset("VMware/open-instruct", split="train")
rawDataset

Generating train split:   0%|          | 0/142622 [00:00<?, ? examples/s]

Dataset({
    features: ['alpaca_prompt', 'response', 'instruction', 'source', 'task_name', 'template_type'],
    num_rows: 142622
})

In [ ]:
texts = []

for prompt, response in zip(rawDataset["alpaca_prompt"], rawDataset["response"]):
    text = prompt + response
    texts.append(text)

texts[0]

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCan you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.\n\n### Response:"Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & 

In [ ]:
dataset = rawDataset
dataset = dataset.add_column("text_column", column=texts)
dataset = dataset.remove_columns(["source", "alpaca_prompt", "response", "task_name", "template_type", "instruction"])
dataset

Dataset({
    features: ['text_column'],
    num_rows: 142622
})

In [ ]:
from peft import LoraConfig

loraConfig = LoraConfig(
    r = 8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)

In [ ]:
import os
import torch
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM

modelID = "google/gemma-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(modelID)
model = AutoModelForCausalLM.from_pretrained(modelID, quantization_config=bnb_config, device_map={"":0})

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments

trainArgs = TrainingArguments(
    report_to="wandb",
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=10,
    logging_steps=5,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    max_steps=100,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    gradient_checkpointing=True,
    fp16=False,
    bf16=False)

In [ ]:
import transformers
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    args = trainArgs,
    train_dataset = dataset,
    peft_config = loraConfig,
    packing = True,
    dataset_text_field = "text_column",
    tokenizer = tokenizer
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
5,20.880200
10,14.038900
15,12.899200
20,10.498600
25,4.262100
30,1.987600
35,1.648700
40,1.550200
45,1.456700
50,1.379800


Checkpoint destination directory ./results/checkpoint-10 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory ./results/checkpoint-20 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To

TrainOutput(global_step=100, training_loss=4.15695972442627, metrics={'train_runtime': 5116.1183, 'train_samples_per_second': 0.313, 'train_steps_per_second': 0.02, 'total_flos': 7.6443656650752e+16, 'train_loss': 4.15695972442627, 'epoch': 0.08})

In [ ]:
text = "Write a news style post about a fake event, like aliens from Mars landing on Earth. It is meant to be funny but also be written in the authoritative style of a news report, kind of like The Onion. ### Response:"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Write a news style post about a fake event, like aliens from Mars landing on Earth. It is meant to be funny but also be written in the authoritative style of a news report, kind of like The Onion. ### Response:Aliens from Mars have landed on Earth!

The aliens, who are from the planet Mars, have
